# 3D Tensor
- color image : 3D (w, h, channel)
- channel : R, G, B (YUV, LUV, )
    - 1 channel image : Grayscale Image(흑백)
    - batch_size * 1 channel Image ==> 3D

In [66]:
import os
import time
import numpy as np
from PIL import Image
from glob import glob

from torch import cuda

In [22]:
path = "d:/Kamie/dataset/dataset_CatDog/"
train_path = os.path.join(path, "training_set")
test_path = os.path.join(path, "test_set")

In [41]:
dog1 = np.array(Image.open(os.path.join(path, "dog.566.jpg")).resize((224, 224)))

In [42]:
dog_tensor = torch.from_numpy(dog1)

In [43]:
print("dog_tensor.size : ", dog_tensor.size())
print("dog_tensor.dim : ", dog_tensor.dim())

dog_tensor.size :  torch.Size([224, 224, 3])
dog_tensor.dim :  3


### 4D tensor : color images가 여러개 들어가 있는 데이터셋


## ```glob``` : 특정 문자를 포함한 모든 것을 가지고 오기
glob 결과는 리스트로 반환

In [38]:
dogs_path = os.path.join(train_path, "dogs")
dogs_path

'd:/Kamie/dataset/dataset_CatDog/training_set\\dogs'

In [39]:
dogs = glob(dogs_path + "/*.jpg")  # 64 x 224 x 224 x 3
dogs[:2]

['d:/Kamie/dataset/dataset_CatDog/training_set\\dogs\\dog.1.jpg',
 'd:/Kamie/dataset/dataset_CatDog/training_set\\dogs\\dog.10.jpg']

In [46]:
img_list = []
for dog in dogs[:64]:
    img_list.append(np.array(Image.open(dog).resize((224, 224))))

# comprehension 해보기
# img_list = [np.array(Image.open(dog).resize((224, 224))) for dog in dogs[:64]]

In [47]:
img_list[0]

array([[[220, 207, 193],
        [143, 124, 105],
        [144, 114,  88],
        ...,
        [131, 101,  81],
        [115,  85,  65],
        [112,  82,  62]],

       [[228, 207, 186],
        [146, 117,  91],
        [148, 110,  74],
        ...,
        [ 88,  56,  34],
        [ 83,  51,  29],
        [ 84,  52,  30]],

       [[227, 202, 179],
        [131,  97,  67],
        [128,  84,  45],
        ...,
        [115,  84,  56],
        [111,  79,  52],
        [102,  70,  42]],

       ...,

       [[254, 252, 241],
        [249, 245, 233],
        [243, 240, 223],
        ...,
        [234, 237, 214],
        [230, 233, 210],
        [231, 233, 211]],

       [[255, 254, 242],
        [250, 249, 234],
        [246, 244, 226],
        ...,
        [238, 239, 220],
        [236, 237, 218],
        [236, 238, 219]],

       [[255, 255, 244],
        [253, 255, 241],
        [251, 252, 236],
        ...,
        [248, 249, 235],
        [246, 247, 233],
        [245, 246, 233]]

In [50]:
# 최종적으로 np.array 4D 형식 만들어주기
dogs_imgs = np.array(img_list)  

print("dogs_imgs.size : ", dogs_imgs.size)  # 224*224*3 * 64장 = 9633792
print("dogs_imgs.shape : ", dogs_imgs.shape)
print("dogs_imgs.ndim : ", dogs_imgs.ndim)

dogs_imgs.size :  9633792
dogs_imgs.shape :  (64, 224, 224, 3)
dogs_imgs.ndim :  4


In [52]:
224 * 224 * 3 * 64

9633792

꼭 넘파이->텐서 할 필요 없이 float tensor로 바로 변환도 가능.  
근데 넘파이랑 호환성 좋아서 같이 많이 쓰는 편

In [53]:
dogs_tensor = torch.from_numpy(dogs_imgs)

print("dogs_tensor.size : ", dogs_tensor.size())  # 224*224*3 * 64장 = 9633792
print("dogs_tensor.ndim : ", dogs_tensor.dim())

dogs_tensor.size :  torch.Size([64, 224, 224, 3])
dogs_tensor.shape :  torch.Size([64, 224, 224, 3])
dogs_tensor.ndim :  4


---
# < pytorch에서 자주 사용되는 함수 >
내가 사용하지 않더라도 어떤 의미인지는 알아야 함

## ▶ ```Broadcasting``` : 자동으로 크기 맞춰서 연산 수행
- matrix A, B를 덧셈/뺄셈 할 땐 두 matrix의 크기가 같아야 함
- 행렬의 곱 : A matrix의 col 크기와 B matrix의 row 크기가 같아야 함

In [55]:
# 1x2 행렬 : [[1,2]] 2D 형태로 표현
m1 = torch.FloatTensor([[3, 3]])  # 1x2
m2 = torch.FloatTensor([[2, 2]])  # 1x2
m1 + m2  # 1x2

tensor([[5., 5.]])

In [57]:
# 1x1 벡터와 1x2 matrix 연산해보기
m1 = torch.FloatTensor([[3, 3]])  # 1x2
m2 = torch.FloatTensor([[2]])  # 1x1
m1 + m2   # 1x2  # 자동으로 m2를 [[2,2]] 동일한 값으로 확장해서 크기 맞춰줌 : broadcasting

tensor([[5., 5.]])

In [59]:
# 1x2와 2x1 matrix 연산해보기
m1 = torch.FloatTensor([[1, 2]])  # 1x2
m2 = torch.FloatTensor([[3], [4]])  # 2x1
m1 + m2   # 2x2  # broadcasting을 통해 m1, m2 둘 다 2x2로 확장해서 계산

tensor([[4., 5.],
        [5., 6.]])

## ▶ ```in-place``` : copy하지 않고 원본 덮어쓰기
- pandas에서 inplace=True하면 DF 원본값을 바꿔준 것과 같음
- in-place : ```_```(언더바) 사용, 언더바 붙으면 inplace하는 연산인가보다 하면 됨

In [62]:
a = torch.FloatTensor([[1,2],[3,4]])
a.add_(2)   # broadcasting + inplace 연산 수행
a

tensor([[3., 4.],
        [5., 6.]])

## ▶ matrix 곱, elements-wise 곱
1) matrix muliplication : ```tensor.matmul(tensor)```  ex) ```m1.matmul(m2)```
    - matrix 곱 : 일반적인 행렬 곱

2) elements-wise 곱 : ```tensor * tensor```   ex) ```m1*m2```, ```m1.mul(m2)```
    - elements-wise 곱 : 같은 위치의 값끼리 곱하기 ex) (0,0) 위치끼리 곱하기
    - 같은 크기의 매트릭스로 수행해야 함, 만약 다르면 브로드캐스팅으로 알아서 크기 맞춰서 계산됨

In [63]:
# 2x2와 2x1의 matmul : 2x1 matrix
m1 = torch.FloatTensor([[1, 2], [3, 4]])  # 2x2
m2 = torch.FloatTensor([[1], [2]])  # 2x1
m1.matmul(m2)  # 2x1

tensor([[ 5.],
        [11.]])

In [64]:
# 2x2와 2x1의 elements-wise 곱 : 2x1 matrix
m1 = torch.FloatTensor([[1, 2], [3, 4]])  # 2x2
m2 = torch.FloatTensor([[1], [2]])  # 2x1
m1 * m2  # 2x2

tensor([[1., 2.],
        [6., 8.]])

## ▶ 평균 ```tensor.mean()```

In [73]:
t = torch.FloatTensor([[1, 2], [3, 4]])
t.mean()  # (1+2+3+4)/4

tensor(2.5000)

## ▶ torch dim을 기준으로 연산 가능
- pandas sum(axis=0) 처럼. : 0은 row, 1은 column
    - cf) 연산의 기본이 column 단위로 하는 연산이면 0=column, 1=row
    
### dim = 0, 1 인가에 따라 연산을 할 수 있음
- ```dim = 0``` : 첫 번재 차원을 제거한다는 의미.  
    *ex. matrix에서 첫 번째 차원을 제거하면 row 제거* ```matrix=2D=(row, col)```. 
- row를 배제하고 계산하고 싶으면 (dim=1) 옵션 적용

In [75]:
t = torch.FloatTensor([[1, 2], [3, 4]])
t.mean(dim=0)  # column끼리 연산 : (1,3)의 평균, (2,4)의 평균

tensor([2., 3.])

In [77]:
t = torch.FloatTensor([[1, 2], [3, 4]])
t.mean(dim=1)  # row끼리 연산 : (1,2)의 평균, (3,4)의 평균

tensor([1.5000, 3.5000])

In [78]:
t = torch.FloatTensor([[1, 2], [3, 4]])
t.sum(dim=0)  # column끼리 연산 : (1,3)의 합, (2,4)의 합

tensor([4., 6.])

In [80]:
t = torch.FloatTensor([[1, 2], [3, 4]])
t.sum(dim=1)  # row끼리 연산 : (1,2)의 합, (3,4)의 합

tensor([3., 7.])

## ▶ max, argmax
- ```max``` : Tensor들의 원소 값 중에 제일 큰 값을 반환 
- ```argmax``` : Tensor들의 원소 값 중에 제일 큰 값의 "인덱스" 반환  
    ex) softmax 후 가장 높은 값 빼내야 클래스 인식 가능

In [81]:
t = torch.FloatTensor([[1, 2], [3, 4]])
t.max(), t.argmax()

(tensor(4.), tensor(3))

In [111]:
t = torch.FloatTensor([[[11, 2], [3, 4], [88, 7]], 
                       [[5, 6], [7, 8], [7, 888]]])
t.max(dim=1), t.argmax(dim=1)
# max의 indices는 argmax 정보를 같이 출력해주는 거. max값의 인덱스

(torch.return_types.max(
 values=tensor([[ 88.,   7.],
         [  7., 888.]]),
 indices=tensor([[2, 2],
         [1, 2]])),
 tensor([[2, 2],
         [1, 2]]))

In [112]:
t.max(dim=0), t.argmax(dim=0)

(torch.return_types.max(
 values=tensor([[ 11.,   6.],
         [  7.,   8.],
         [ 88., 888.]]),
 indices=tensor([[0, 1],
         [1, 1],
         [0, 1]])),
 tensor([[0, 1],
         [1, 1],
         [0, 1]]))

In [108]:
t = torch.FloatTensor([[9, 8, 77, 6, 5], [31, 2, 5, 10, 4]])
t.max(dim=1), t.argmax(dim=1)

(torch.return_types.max(
 values=tensor([77., 31.]),
 indices=tensor([2, 0])),
 tensor([2, 0]))

In [109]:
t.max(dim=0), t.argmax(dim=0)

(torch.return_types.max(
 values=tensor([31.,  8., 77., 10.,  5.]),
 indices=tensor([1, 0, 0, 1, 0])),
 tensor([1, 0, 0, 1, 0]))

## ▶ view : 원소의 개수를 유지하면서 tensor의 shape을 변경
- 넘파이의 reshape()과 동일한 기능
- 텐서의 크기, 차수를 변경하지만 총 원소의 개수는 변화 없음!

In [86]:
# 2x2x3 tensor 생성 : 총 원소의 개수 = 12
t = torch.FloatTensor(
[[[0, 1, 2],[3, 4, 5]],
 [[6, 7, 8],[9, 10, 11]]]
)
t.shape, t.dim()

(torch.Size([2, 2, 3]), 3)

In [89]:
# 3D -> 2D 텐서로 변경
t2 = t.view([-1, 3])
# -1 : all을 의미
# row = -1 이라는 것은 col의 개수에 따라 row의 형태는 알아서 계산하라는 의미
# col = 3 으로 고정했으므로 row를 알아서 4로 계산. (t 원소가 총 12개였으므로.)
t2.shape, t2

(torch.Size([4, 3]),
 tensor([[ 0.,  1.,  2.],
         [ 3.,  4.,  5.],
         [ 6.,  7.,  8.],
         [ 9., 10., 11.]]))

In [116]:
# 3차원 텐서의 크기만 변경
# 2x2x3 -> ?x1x3 -> t.view([-1, 1, 3]) : ?=4
t = torch.FloatTensor(
[[[0, 1, 2],[3, 4, 5]],
 [[6, 7, 8],[9, 10, 11]]]
)

t.view([-1, 1, 3])

tensor([[[ 0.,  1.,  2.]],

        [[ 3.,  4.,  5.]],

        [[ 6.,  7.,  8.]],

        [[ 9., 10., 11.]]])

## ▶ squeeze <-> unsqueeze
- 2x1 matrix를 원소가 2개인 벡터 형태로 변경

In [118]:
t = torch.FloatTensor([[0], [1], [2]])
t.squeeze().shape, t.squeeze()

(torch.Size([3]), tensor([0., 1., 2.]))

In [125]:
t = torch.FloatTensor([0, 1, 2])
t_row = t.unsqueeze(dim=0)  # 옵션에 0 주면 row에 1 추가. 1x3  t.unsqueeze(0) 해도 됨 
t_col = t.unsqueeze(dim=1)  # col에 1 추가. 3x1

print("origin : ", t.shape, t)
print("dim=0 : ", t_row.shape, t_row)
print("dim=1 : ", t_col.shape, t_col)

origin :  torch.Size([3]) tensor([0., 1., 2.])
dim=0 :  torch.Size([1, 3]) tensor([[0., 1., 2.]])
dim=1 :  torch.Size([3, 1]) tensor([[0.],
        [1.],
        [2.]])


In [120]:
# tensor는 안됨
t = torch.FloatTensor([[[0, 1], [1, 2]],[[3, 4], [5, 6]]])
t.squeeze().shape, t.squeeze()

(torch.Size([2, 2, 2]),
 tensor([[[0., 1.],
          [1., 2.]],
 
         [[3., 4.],
          [5., 6.]]]))

## ▶ concatenate 두 개의 텐서를 연결해서 하나의 텐서 생성
```python
torch.cat()
```

In [131]:
# 2x2 텐서를 생성하고 연결
x = torch.FloatTensor([[1, 2], [3, 4]])
y = torch.FloatTensor([[5, 6], [7, 8]])

row = torch.cat([x, y], dim=0)
col = torch.cat([x, y], dim=1)

print(f"dim=0 : {row.shape},\n\t{row}")
print(f"dim=1 : {col.shape},\n\t{col}")

dim=0 : torch.Size([4, 2]),
	tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]])
dim=1 : torch.Size([2, 4]),
	tensor([[1., 2., 5., 6.],
        [3., 4., 7., 8.]])


---
# GPU

In [65]:
# GPU 사용 가능 여부 확인하기
cuda.is_available()

True

In [67]:
# 사용 가능한 GPU 개수 확인하기
torch.cuda.device_count()

1

In [133]:
a = torch.rand(20000, 20000)
b = torch.rand(20000, 20000)

In [134]:
# CPU 사용
start_time = time.time()

a.matmul(b)
    
end_time = time.time()
end_time - start_time  # 소요 시간 측정

19.82026433944702

In [71]:
# GPU 사용
start_time = time.time()

if cuda.is_available():
    a = a.cuda()
    b = b.cuda()
    a.matmul(b)
    
end_time = time.time()
end_time - start_time  # 소요 시간 측정

3.3217294216156006